In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import scipy.stats 
import scipy.integrate as integrate
import matplotlib.pyplot as plt

The purpose of this notebook is to create generalized/customizable functions that can be used for Large Scale Food Fortification multiplication models with dichotomous outcomes (zinc, vitamin A, folic acid). The outcomes (DALYs averted) generated by this notebook assume the following:

- Complete scale-up achieved between starting baseline and alternative scenario coverage (med/high/low levels), defined according to the proportion of the population that eats industrially produced vehicles. This notebook does NOT currently consider the additional coverage over time in the alternative scenario defined according to the proportion of the population that eats the vehicle at all (due to campaign to convince additional individuals to eat fortified versions of vehicle).
- All individuals covered by fortification are assumed to be *effectively* covered. This assumption is not valid based on age- and timing-effects built into the full-scale models. These nutrient-specific effects should be added into the respective mutliplication model for the full results

In [2]:
location_ids = [163, 214, 205, 190, 189]

"""Note: full set of location IDs is shown below, but subset used here
was selected because they are the locations with non-missing coverage data
for the nutrient and vehicle of interest (vitamin A/oil)

[168, 161, 201, 202, 6, 205, 171, 141, 179, 207, 163, 11, 180, 181,
184, 15, 164, 213, 214, 165, 196, 522, 190, 189, 20]"""

ages = [2,3,4,5]
sexes = [1,2]

index_cols=['location_id','sex_id','age_group_id']

# define alternative scenario coverage levels (low, medium, high)
    # this parameter represents the proportion of additional coverage achieved in the
    # alternative scenario, defined as the difference between the proportion of the population
    # that eats the fortified vehicle and the proportion of the population that eats 
    # the industrially produced vehicle
alternative_scenario_coverage_levels = [0.25, 0.5, 0.75]

In [3]:
# vitamin A specific -- these should be replaced for other models
rei_id = 96
cause_ids = [389, 302, 341]
nonfatal_causes = [389]
nutrient = 'vitamin a'
vehicle = 'oil'

In [4]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 2.22
q_975 = 5.26

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)
rr_distribution = lognorm(s=sigma, scale=median)

In [66]:
def generate_rr_deficiency_nofort_draws(mean, std):
    """This function takes a distribution for the relative risk
    for lack of fortification of a particular nutrient and generates
    1,000 draws based on that distribution. The data is the duplicated
    so that it is the same for each location ID so that it can be easily
    used later in the calculations."""
    data = pd.DataFrame()    
    np.random.seed(7)
    data['rr'] = np.random.lognormal(mean, std, size=1000)
    draws = []
    for i in list(range(0,1000)):
        draws.append(f'draw_{i}')
    data['draws'] = draws
    data = pd.DataFrame.pivot_table(data, values='rr', columns='draws').reset_index().drop(columns=['index'])
    df = pd.DataFrame(np.repeat(data.values,len(location_ids),axis=0))
    df.columns = data.columns
    df['location_id'] = location_ids
    df = df.set_index('location_id')
    return df

In [6]:
def pull_cause_specific_dalys_deficiency_pafs(rei_id, cause_ids):
    """This function pulls PAF data from GBD for specified 
    risk outcome pairs. Note that the risk in this context 
    will/should be nutrient *deficiencies*, not the lack of 
    nutrient fortification"""
    
    data = pd.DataFrame()
    for cause_id in cause_ids:
        temp = get_draws(
            gbd_id_type=['rei_id', 'cause_id'], 
            gbd_id=[rei_id, cause_id],
            source='burdenator',
            measure_id=2, #dalys
            metric_id=2, #percent
            location_id=location_ids,
            year_id=2019,
            age_group_id=ages,
            sex_id=sexes, 
            gbd_round_id=6,
            status='best',
            decomp_step='step5',
        )
        data = pd.concat([data,temp], ignore_index=True)
    data = data.set_index(index_cols + ['cause_id'])
    data = data.drop(columns=[c for c in data.columns if 'draw' not in c]).sort_index()
    return data

In [7]:
def pull_dalys(cause_ids):
    """This function pulls dalys for specified cause IDs from GBD"""
    
    ylds = get_draws(
        gbd_id_type='cause_id', 
        gbd_id=cause_ids,
        source='como',
        measure_id=3,
        metric_id=3, # only available as rate
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes, 
        gbd_round_id=6,
        status='best',
        decomp_step='step5',
    ).set_index(index_cols + ['cause_id'])
    ylds = ylds.drop(columns=[c for c in ylds.columns if 'draw' not in c])
    pop = get_population(
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes,
        gbd_round_id=6,
        decomp_step='step4').set_index(index_cols)
    for i in list(range(0,1000)):
        ylds[f'draw_{i}'] = ylds[f'draw_{i}'] * pop['population']
    ylls = get_draws(
        gbd_id_type='cause_id', 
        gbd_id=cause_ids,
        source='codcorrect',
        measure_id=4,
        metric_id=1, 
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes, 
        gbd_round_id=6,
        status='latest',
        decomp_step='step5',
    ).set_index(index_cols + ['cause_id']).replace(np.nan, 0)
    ylls= ylls.drop(columns=[c for c in ylls.columns if 'draw' not in c])
    for nf in nonfatal_causes:
        nonfatal = ylls.groupby(index_cols).sum()
        nonfatal['cause_id'] = nf
        for i in list(range(0,1000)):
            nonfatal[f'draw_{i}'] = 0
    ylls = pd.concat([ylls.reset_index(), nonfatal.reset_index()]).set_index(index_cols + ['cause_id'])
    
    dalys = ylls + ylds
    return dalys

In [67]:
def load_coverage_data(nutrient, vehicle):
    data = pd.read_csv('/ihme/homes/alibow/notebooks/vivarium_data_analysis/pre_processing/lsff_project/data_prep/outputs/LSFF_extraction_clean_data_rich_locations_01_11_2021.csv')
    alpha = (data.loc[data.vehicle == vehicle]
             .loc[data.nutrient == nutrient]
             .loc[data.value_description == 'percent of population eating fortified vehicle'])
    alpha_star = (data.loc[data.vehicle == vehicle]
                  .loc[data.value_description == 'percent of population eating industrially produced vehicle'])

    
    # generate draws
    """This currently relies on two major assumptions:
    1. Truncated normal distribution
    2. The same percentile from the eats_fortified and eats_fortifiable distributions sampled for each draw
    
    Assumption number two is likely overly restrictive, but was chosen such that eats_fortified will 
    always be less than eats_fortifiable at the draw level (this is consistent with methodology described
    in 2017 concept model, but is achieved by setting the same random seed to sample each of these
    parameters)"""
      
    for data in [alpha, alpha_star]:
              
        data['value_std'] = (data.value_975_percentile - data.value_025_percentile) / 2 / 1.96
        data['a'] = (data.value_025_percentile - data.value_mean) / data.value_std
        data['b'] = (data.value_975_percentile - data.value_mean) / data.value_std       
        np.random.seed(11)
        for i in list(range(0,1000)):
            data[f'draw_{i}'] = scipy.stats.truncnorm.rvs(data.a, data.b, data.value_mean, data.value_std) / 100
            
    alpha = (alpha.set_index('location_id')
         .drop(columns=[c for c in alpha.columns if 'draw' not in c and c != 'location_id']))
    alpha_star = (alpha_star.set_index('location_id')
         .drop(columns=[c for c in alpha_star.columns if 'draw' not in c and c != 'location_id']))
    alpha_star_low = (alpha_star - alpha) * alternative_scenario_coverage_levels[0] + alpha
    alpha_star_low['coverage_level'] = 'low'
    alpha_star_med = (alpha_star - alpha) * alternative_scenario_coverage_levels[1] + alpha
    alpha_star_med['coverage_level'] = 'medium'
    alpha_star_high = (alpha_star - alpha) * alternative_scenario_coverage_levels[2] + alpha
    alpha_star_high['coverage_level'] = 'high'
    alpha_star_full = alpha_star.copy()
    alpha_star_full['coverage_level'] = 'full'
    
    alpha_star = pd.concat([alpha_star_low.reset_index(), 
                            alpha_star_med.reset_index(), 
                            alpha_star_high.reset_index(),
                            alpha_star_full.reset_index()], 
                           ignore_index=True)
    alpha_star = alpha_star.set_index([c for c in alpha_star.columns if 'draw' not in c])
    
    #p = 1 - alpha
    #p_star = 1 - alpha_star
    
    return alpha, alpha_star

In [40]:
def load_alternative_coverage_data(nutrient, vehicle):
    data = pd.read_csv('/ihme/homes/alibow/notebooks/vivarium_data_analysis/pre_processing/lsff_project/data_prep/outputs/LSFF_extraction_clean_data_rich_locations_01_11_2021.csv')
    alpha = (data.loc[data.vehicle == vehicle]
             .loc[data.nutrient == nutrient]
             .loc[data.value_description == 'percent of population eating fortified vehicle'])
    alpha_star = (data.loc[data.vehicle == vehicle]
                  .loc[data.value_description == 'percent of population eating industrially produced vehicle'])

    
    # generate draws
    """This currently relies on two major assumptions:
    1. Truncated normal distribution
    2. The same percentile from the eats_fortified and eats_fortifiable distributions sampled for each draw
    
    Assumption number two is likely overly restrictive, but was chosen such that eats_fortified will 
    always be less than eats_fortifiable at the draw level (this is consistent with methodology described
    in 2017 concept model, but is achieved by setting the same random seed to sample each of these
    parameters)"""
      
    for data in [alpha, alpha_star]:
              
        data['value_std'] = (data.value_975_percentile - data.value_025_percentile) / 2 / 1.96
        data['a'] = (data.value_025_percentile - data.value_mean) / data.value_std
        data['b'] = (data.value_975_percentile - data.value_mean) / data.value_std       
        np.random.seed(1246)
        for i in list(range(0,1000)):
            data[f'draw_{i}'] = scipy.stats.truncnorm.rvs(data.a, data.b, data.value_mean, data.value_std) / 100
            
    alpha = (alpha.set_index('location_id')
         .drop(columns=[c for c in alpha.columns if 'draw' not in c and c != 'location_id']))
    alpha_star = (alpha_star.set_index('location_id')
         .drop(columns=[c for c in alpha_star.columns if 'draw' not in c and c != 'location_id']))
    alpha_star_low = (alpha_star) * alternative_scenario_coverage_levels[0]
    alpha_star_low['coverage_level'] = 'low'
    alpha_star_med = (alpha_star) * alternative_scenario_coverage_levels[1]
    alpha_star_med['coverage_level'] = 'medium'
    alpha_star_high = (alpha_star) * alternative_scenario_coverage_levels[2]
    alpha_star_high['coverage_level'] = 'high'
    
    alpha_star = pd.concat([alpha_star_low.reset_index(), 
                            alpha_star_med.reset_index(), 
                            alpha_star_high.reset_index()], 
                           ignore_index=True)
    alpha_star = alpha_star.set_index([c for c in alpha_star.columns if 'draw' not in c])
    
    #p = 1 - alpha
    #p_star = 1 - alpha_star
    
    return alpha, alpha_star

In [10]:
def calculate_paf_deficiency_nofort(rr_deficiency_nofort, alpha):
    """This function calculates the population attributable fraction of UNfortified food
    on the fortification outcome of interest (outcome defined in the fortification 
    effect size, which is generally nutrient deficiency)
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
       
    paf_deficiency_nofort = ((rr_deficiency_nofort - 1) * (1 - alpha)) / ((rr_deficiency_nofort - 1) * (1 - alpha) + 1)
    return paf_deficiency_nofort

In [11]:
def calculate_pif_deficiency_nofort(paf_deficiency_nofort, alpha, alpha_star):
    """This function calculates the population impact fraction for UNfortified 
    foods and nutrient deficiency based on the location-specific coverage
    levels of fortified foods; specifically, p (1 - proportion of population
    that eats fortified vehicle) and p_start (1 - proportion of population that 
    eats industrially produced vehicle).
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
    pif_deficiency_nofort = paf_deficiency_nofort * (alpha_star - alpha) / (1 - alpha)
    return pif_deficiency_nofort

In [61]:
def calculate_daly_reduction_by_cause(pif_deficiency_nofort, paf_dalys_deficiency, dalys):
    """This functionc calculates the population impact fraction for UNfortified 
    food and DALYs due to specific causes as well as the total number of DALYs
    averted by cause, sex, and age
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
    
    df = pd.DataFrame()
    
    for level in ['low','medium','high','full']:
        pif_deficiency_nofort_level = (pif_deficiency_nofort.reset_index()
                                     .loc[pif_deficiency_nofort.reset_index().coverage_level == level]
                                     .drop(columns='coverage_level')
                                     .set_index('location_id'))
        pif_dalys_nofort = pif_deficiency_nofort_level * paf_dalys_deficiency
        pif_dalys_nofort['measure'] = 'pif'
        dalys_reduction = pif_dalys_nofort * dalys
        dalys_reduction['measure'] = 'dalys averted'
        dalys_reduction_overall = dalys_reduction.reset_index().groupby(index_cols + ['measure']).sum().reset_index()
        dalys_reduction_overall['cause_id'] = 294
        data = (pd.concat([pif_dalys_nofort.reset_index(), dalys_reduction.reset_index(), dalys_reduction_overall], ignore_index=True))
        data['coverage_level'] = level
        data = data.set_index(index_cols + ['measure','cause_id','coverage_level']).dropna().sort_index()
        df = pd.concat([df,data])
        
    return df

In [68]:
rr_deficiency_nofort = generate_rr_deficiency_nofort_draws(mu, sigma)
rr_deficiency_nofort.mean(axis=1)

location_id
163    2.396239
214    2.396239
205    2.396239
190    2.396239
189    2.396239
dtype: float64

In [14]:
paf_dalys_deficiency = pull_cause_specific_dalys_deficiency_pafs(rei_id, cause_ids)
paf_dalys_deficiency.head()

draw_0    draw_1   draw_10  \
location_id sex_id age_group_id cause_id                                 
163         1      2            389       1.000000  1.000000  1.000000   
                   3            389       1.000000  1.000000  1.000000   
                   4            302       0.023614  0.022761  0.032084   
                                341       0.062437  0.068525  0.080122   
                                389       1.000000  1.000000  1.000000   

                                          draw_100  draw_101  draw_102  \
location_id sex_id age_group_id cause_id                                 
163         1      2            389       1.000000  1.000000  1.000000   
                   3            389       1.000000  1.000000  1.000000   
                   4            302       0.061713  0.010529  0.016051   
                                341       0.158759  0.013909  0.026354   
                                389       1.000000  1.000000  1.000000   

                                          draw_103  draw_104  draw_105  \
location_id sex_id age_group_id cause_id                                 
163         1      2            389       1.000000  1.000000  1.000000   
                   3            389       1.000000  1.000000  1.000000   
                   4            302       0.008245  0.045553  0.027902   
                                341       0.011891  0.113604  0.067258   
                                389       1.000000  1.000000  1.000000   

                                          draw_106  ...  draw_990  draw_991  \
location_id sex_id age_group_id cause_id            ...                       
163         1      2            389       1.000000  ...  1.000000  1.000000   
                   3            389       1.000000  ...  1.000000  1.000000   
                   4            302       0.032493  ...  0.013291  0.040655   
                                341       0.091411  ...  0.028041  0.109343   
                                389       1.000000  ...  1.000000  1.000000   

                                          draw_992  draw_993  draw_994  \
location_id sex_id age_group_id cause_id                                 
163         1      2            389       1.000000  1.000000  1.000000   
                   3            389       1.000000  1.000000  1.000000   
                   4            302       0.013134  0.040318  0.050091   
                                341       0.039931  0.107498  0.148924   
                                389       1.000000  1.000000  1.000000   

                                          draw_995  draw_996  draw_997  \
location_id sex_id age_group_id cause_id                                 
163         1      2            389       1.000000  1.000000  1.000000   
                   3            389       1.000000  1.000000  1.000000   
                   4            302       0.018375  0.019796  0.037472   
                                341       0.044538  0.046736  0.096353   
                                389       1.000000  1.000000  1.000000   

                                          draw_998  draw_999  
location_id sex_id age_group_id cause_id                      
163         1      2            389       1.000000  1.000000  
                   3            389       1.000000  1.000000  
                   4            302       0.025155  0.009430  
                                341       0.066101  0.021711  
                                389       1.000000  1.000000  

[5 rows x 1000 columns]

In [15]:
dalys = pull_dalys(cause_ids)
dalys.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v104/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



draw_0         draw_1  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       228052.286761  256459.736242   
                                341            0.000000       0.000000   
                                389          249.085991     270.466085   
                   3            302       207609.552344  263078.492814   
                                341            0.000000       0.000000   

                                                draw_10       draw_100  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       146290.339705  192676.381120   
                                341            0.000000       0.000000   
                                389          270.284907     146.168022   
                   3            302       198861.237612  173638.640755   
                                341            0.000000       0.000000   

                                               draw_101       draw_102  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       172362.436243  187454.984499   
                                341            0.000000       0.000000   
                                389          118.041018     158.115471   
                   3            302       255710.254455  184070.329711   
                                341            0.000000       0.000000   

                                               draw_103       draw_104  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       159534.221668  176235.755329   
                                341            0.000000       0.000000   
                                389          160.563119     111.986426   
                   3            302       167459.228561  282709.744663   
                                341            0.000000       0.000000   

                                               draw_105       draw_106  ...  \
location_id sex_id age_group_id cause_id                                ...   
163         1      2            302       170489.703795  151586.434309  ...   
                                341            0.000000       0.000000  ...   
                                389          347.919477     159.200861  ...   
                   3            302       228526.429634  190682.116226  ...   
                                341            0.000000       0.000000  ...   

                                               draw_990       draw_991  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       233728.203639  195240.822126   
                                341            0.000000       0.000000   
                                389           87.646759     195.252324   
                   3            302       266360.552338  251853.188296   
                                341            0.000000       0.000000   

                                               draw_992       draw_993  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       264767.777947  189854.851927   
                                341            0.000000       0.000000   
                                389          129.934700     254.856255   
                   3            302       250290.066661  189621.733181   
                                341            0.000000       0.000000   

                                               draw_994       draw_995  \
location_id sex_id age_group_id cause_id                                 
163         1      2            302       201259.960318  246817.767701   
                                341            0.000000       0.000000   
                                389           47.148614     134.662517   
           

In [69]:
alpha, alpha_star = load_coverage_data(nutrient, vehicle)
alpha.mean(axis=1)

location_id
163    0.243901
214    0.074497
205    0.980048
190    0.543213
189    0.537420
dtype: float64

In [70]:
paf_deficiency_nofort = calculate_paf_deficiency_nofort(rr_deficiency_nofort, alpha)
paf_deficiency_nofort.mean(axis=1)

location_id
163    0.443970
214    0.485210
205    0.026645
190    0.342059
189    0.344187
dtype: float64

In [71]:
pif_deficiency_nofort = calculate_pif_deficiency_nofort(paf_deficiency_nofort, alpha, alpha_star)
pif_deficiency_nofort.mean(axis=1).head()

location_id  coverage_level
163          low               0.095484
214          low               0.031349
205          low               0.000000
190          low               0.064860
189          low               0.072513
dtype: float64

In [72]:
daly_reduction = calculate_daly_reduction_by_cause(pif_deficiency_nofort, paf_dalys_deficiency, dalys)
daly_reduction.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v104/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



draw_0  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              39.073737   
                                              389      low              39.073737   
                                pif           389      low               0.156868   
                   3            dalys averted 294      low             522.240431   
                                              389      low             522.240431   

                                                                           draw_1  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              22.067716   
                                              389      low              22.067716   
                                pif           389      low               0.081591   
                   3            dalys averted 294      low             214.110377   
                                              389      low             214.110377   

                                                                          draw_10  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              19.851739   
                                              389      low              19.851739   
                                pif           389      low               0.073447   
                   3            dalys averted 294      low             152.106619   
                                              389      low             152.106619   

                                                                         draw_100  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low               8.851672   
                                              389      low               8.851672   
                                pif           389      low               0.060558   
                   3            dalys averted 294      low             195.935534   
                                              389      low             195.935534   

                                                                         draw_101  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              14.058790   
                                              389      low              14.058790   
                                pif           389      low               0.119101   
                   3            dalys averted 294      low             302.270450   
                                              389      low             302.270450   

                                                                         draw_102  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              18.749362   
                                              389      low              18.749362   
                                pif           389      low               0.118580   
                   3            dalys averted 294      low             182.141138   
                                              389      low             182.141138   

                                                                        draw_103  \
location_id sex_id age_group_id measure       cause_id coverage_level              
163         1      2            dalys averted 294      low              4.063897   
                                              389      low              4.063897   
                                pif           389      low              0.025310   
                   3            dalys averted 294      low             86.53384

In [73]:
# check and make sure that there are only negative dalys averted for execpted draws
    # (draws with RR for fortification < 1 and draws with negative GBD PAFs)

in_neg_draws = np.concatenate([pd.DataFrame(rr_deficiency_nofort.stack()).loc[pd.DataFrame(rr_deficiency_nofort.stack())[0] < 1].reset_index()['draws'].unique(),
            pd.DataFrame(paf_dalys_deficiency.stack()).loc[pd.DataFrame(paf_dalys_deficiency.stack())[0] < 0].reset_index()['level_4'].unique()])

out_neg_draws = pd.DataFrame(daly_reduction.stack()).reset_index().rename(columns={'level_6':'draw',0:'val'})
out_neg_draws = out_neg_draws.loc[out_neg_draws.val < 0]

assert len([c for c in out_neg_draws.draw.unique() if c not in in_neg_draws]) == 0, "Error: unexpected negative values"

In [74]:
fort_daly_reduction_by_location = daly_reduction.groupby(['location_id','measure','cause_id','coverage_level']).sum().reset_index()
fort_daly_reduction_by_location = (fort_daly_reduction_by_location
                                   .loc[fort_daly_reduction_by_location.measure=='dalys averted']
                                   .loc[fort_daly_reduction_by_location.cause_id==294])
fort_daly_reduction_by_location = (fort_daly_reduction_by_location
                                   .set_index(['location_id','measure','cause_id','coverage_level'])
                                   .apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1))
    
fort_daly_reduction_by_location

count          mean  \
location_id measure       cause_id coverage_level                         
163         dalys averted 294      full            1000.0  96007.743430   
                                   high            1000.0  72005.807572   
                                   low             1000.0  24001.935857   
                                   medium          1000.0  48003.871715   
189         dalys averted 294      full            1000.0  10996.353748   
                                   high            1000.0   8247.265311   
                                   low             1000.0   2749.088437   
                                   medium          1000.0   5498.176874   
190         dalys averted 294      full            1000.0  10646.945736   
                                   high            1000.0   7985.209302   
                                   low             1000.0   2661.736434   
                                   medium          1000.0   5323.472868   
205         dalys averted 294      full            1000.0      0.000000   
                                   high            1000.0      0.000000   
                                   low             1000.0      0.000000   
                                   medium          1000.0      0.000000   
214         dalys averted 294      full            1000.0  29224.528934   
                                   high            1000.0  21918.396700   
                                   low             1000.0   7306.132233   
                                   medium          1000.0  14612.264467   

                                                            std  \
location_id measure       cause_id coverage_level                 
163         dalys averted 294      full            55063.957051   
                                   high            41297.967788   
                                   low             13765.989263   
                                   medium          27531.978525   
189         dalys averted 294      full             8223.412083   
                                   high             6167.559062   
                                   low              2055.853021   
                                   medium           4111.706042   
190         dalys averted 294      full             8731.847671   
                                   high             6548.885754   
                                   low              2182.961918   
                                   medium           4365.923836   
205         dalys averted 294      full                0.000000   
                                   high                0.000000   
                                   low                 0.000000   
                                   medium              0.000000   
214         dalys averted 294      full            20276.616828   
                                   high            15207.462621   
                                   low              5069.154207   
                                   medium          10138.308414   

                                                             min         2.5%  \
location_id measure       cause_id coverage_level                               
163         dalys averted 294      full           -108195.163728 -4373.326251   
                                   high            -81146.372796 -3279.994688   
                                   low             -27048.790932 -1093.331563   
                                   medium          -54097.581864 -2186.663125   
189         dalys averted 294      full             -8241.766210  -540.133729   
                                   high             -6181.324658  -405.100297   
                                   low              -2060.441553  -135.033432   
                                   medium           -4120.883105  -270.066865   
190         dalys averted 294      full             -7617.963381  -501.836712   
                                   

In [76]:
pop = (get_population(location_id=location_ids,
                    sex_id=sexes,
                    age_group_id=ages,
                    gbd_round_id=6,
                    decomp_step='step4')
       .groupby('location_id').sum())
pop = pop[['population']]
rates = pop.reset_index().merge(fort_daly_reduction_by_location.reset_index(), on='location_id')
for col in ['mean','2.5%','97.5%']:
    rates[f'{col}'] = rates[f'{col}'] / rates.population * 100_000
rates['measure'] = 'dalys averted per 100,000'
rates = rates[['location_id','coverage_level','measure','cause_id','mean','2.5%','97.5%']]
rates


# NOTE: the 'full' coverage level can be used to compare to previous simulation results (from 2017 data) in 2021, specifically

,location_id,coverage_level,measure,cause_id,mean,2.5%,97.5%
0,163,full,"dalys averted per 100,000",294,82.008302,-3.735626,176.355316
1,163,high,"dalys averted per 100,000",294,61.506226,-2.801720,132.266487
2,163,low,"dalys averted per 100,000",294,20.502075,-0.933907,44.088829
3,163,medium,"dalys averted per 100,000",294,41.004151,-1.867813,88.177658
4,189,full,"dalys averted per 100,000",294,118.147916,-5.803349,328.290086
5,189,high,"dalys averted per 100,000",294,88.610937,-4.352511,246.217565
6,189,low,"dalys averted per 100,000",294,29.536979,-1.450837,82.072522
7,189,medium,"dalys averted per 100,000",294,59.073958,-2.901674,164.145043
8,190,full,"dalys averted per 100,000",294,150.173786,-7.078342,450.955168
9,190,high,"dalys averted per 100,000",294,112.630340,-5.308756,338.216376


# Alternative target coverage rates

These coverage rates represent low/med/high coverage of 25/50/75% of the population that eats the industrially produced vehicle, as opposed to low/med/high coverage of 25/50/75% of the difference between the population that eats fortified vehicle and the population that eats the industrially produced vehicle, as shown above

In [23]:
alpha_alt, alpha_star_alt = load_alternative_coverage_data(nutrient, vehicle)

In [24]:
paf_deficiency_nofort_alt = calculate_paf_deficiency_nofort(rr_deficiency_nofort, alpha_alt)
paf_deficiency_nofort_alt.mean(axis=1)

location_id
163    0.452099
214    0.492045
205    0.028645
190    0.350497
189    0.353362
dtype: float64

In [25]:
pif_deficiency_nofort_alt = calculate_pif_deficiency_nofort(paf_deficiency_nofort_alt, alpha_alt, alpha_star_alt)
pif_deficiency_nofort_alt.mean(axis=1).head()

location_id  coverage_level
163          low              -0.011860
214          low               0.001946
205          low              -1.040172
190          low              -0.248323
189          low              -0.234609
dtype: float64

In [26]:
daly_reduction_alt = calculate_daly_reduction_by_cause(pif_deficiency_nofort_alt, paf_dalys_deficiency, dalys)
daly_reduction.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v104/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



draw_0  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              27.792694   
                                              389      low              27.792694   
                                pif           389      low               0.111579   
                   3            dalys averted 294      low             371.463538   
                                              389      low             371.463538   

                                                                           draw_1  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              19.965065   
                                              389      low              19.965065   
                                pif           389      low               0.073817   
                   3            dalys averted 294      low             193.709559   
                                              389      low             193.709559   

                                                                          draw_10  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              39.767050   
                                              389      low              39.767050   
                                pif           389      low               0.147130   
                   3            dalys averted 294      low             304.700338   
                                              389      low             304.700338   

                                                                         draw_100  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              15.976074   
                                              389      low              15.976074   
                                pif           389      low               0.109299   
                   3            dalys averted 294      low             353.637208   
                                              389      low             353.637208   

                                                                         draw_101  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              14.242039   
                                              389      low              14.242039   
                                pif           389      low               0.120653   
                   3            dalys averted 294      low             306.210386   
                                              389      low             306.210386   

                                                                         draw_102  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              15.624321   
                                              389      low              15.624321   
                                pif           389      low               0.098816   
                   3            dalys averted 294      low             151.782856   
                                              389      low             151.782856   

                                                                         draw_103  \
location_id sex_id age_group_id measure       cause_id coverage_level               
163         1      2            dalys averted 294      low              24.087436   
                                              389      low              24.087436   
                                pif           389      low               0.150018   
                   3            dalys averted 294      low             512

In [27]:
fort_daly_reduction_by_location_alt = daly_reduction_alt.groupby(['location_id','measure','cause_id','coverage_level']).sum().reset_index()
fort_daly_reduction_by_location_alt = (fort_daly_reduction_by_location_alt
                                   .loc[fort_daly_reduction_by_location_alt.measure=='dalys averted']
                                   .loc[fort_daly_reduction_by_location_alt.cause_id==294])
fort_daly_reduction_by_location_alt = (fort_daly_reduction_by_location_alt
                                   .set_index(['location_id','measure','cause_id','coverage_level'])
                                   .apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1))
    
fort_daly_reduction_by_location_alt

count          mean  \
location_id measure       cause_id coverage_level                         
163         dalys averted 294      high            1000.0  63698.277249   
                                   low             1000.0  -2998.398830   
                                   medium          1000.0  30349.939210   
189         dalys averted 294      high            1000.0   4514.399132   
                                   low             1000.0  -8821.793472   
                                   medium          1000.0  -2153.697170   
190         dalys averted 294      high            1000.0   3823.039783   
                                   low             1000.0 -10029.206412   
                                   medium          1000.0  -3103.083315   
205         dalys averted 294      high            1000.0  -6675.954489   
                                   low             1000.0 -20027.863466   
                                   medium          1000.0 -13351.908977   
214         dalys averted 294      high            1000.0  19712.049134   
                                   low             1000.0    451.181557   
                                   medium          1000.0  10081.615346   

                                                            std  \
location_id measure       cause_id coverage_level                 
163         dalys averted 294      high            36565.338646   
                                   low              2717.383224   
                                   medium          17476.169920   
189         dalys averted 294      high             3380.473244   
                                   low              6800.755319   
                                   medium           1899.342754   
190         dalys averted 294      high             3131.370659   
                                   low              8280.211693   
                                   medium           2652.524186   
205         dalys averted 294      high             5885.348903   
                                   low             17656.046710   
                                   medium          11770.697806   
214         dalys averted 294      high            13678.213615   
                                   low               713.413208   
                                   medium           7011.473402   

                                                             min  \
location_id measure       cause_id coverage_level                  
163         dalys averted 294      high           -130076.206117   
                                   low             -15414.106073   
                                   medium          -64509.805416   
189         dalys averted 294      high             -4283.151808   
                                   low             -52573.791677   
                                   medium          -16361.546540   
190         dalys averted 294      high             -2814.544844   
                                   low             -66346.624788   
                                   medium          -21615.313534   
205         dalys averted 294      high            -46513.542987   
                                   low            -139540.628960   
                                   medium          -93027.085973   
214         dalys averted 294      high            -50341.607376   
                                   low              -1435.681830   
                                   medium          -25879.131461   

                                                           2.5%           50%  \
location_id measure       cause_id coverage_level                               
163         dalys averted 294      high           -10144.584344  63187.314560   
                                   low             -9653.129153  -2471.498636   
                                   medium          -4796.661107  30197.258677   
189         dalys averted 294      high             -549.340880   3909.575